In [ ]:
import weave
import weave.legacy.weave.panels

In [ ]:
from weave.legacy.scripts import syndata

In [ ]:
data = weave.save(syndata.random_metrics(100, 100))

In [ ]:
import weave
from weave.legacy.weave import ops_arrow
from weave.legacy.weave.ecosystem import wandb
import datetime
import random
random.seed(1)

min_x = datetime.datetime(2023, 3, 22, 11, 0, 0)
max_x = min_x + datetime.timedelta(hours=1)

users = ['nick', 'shawn', 'stacey', 'tim', 'danny']
model_versions = ['v1', 'v2']

num_requests = 100

def random_datetime_in_range() -> datetime.datetime:
    min_timestamp = min_x.timestamp()
    max_timestamp = max_x.timestamp()
    random_timestamp = random.uniform(min_timestamp, max_timestamp)
    return datetime.datetime.fromtimestamp(random_timestamp)

def random_suggestion(n: int) -> str:
    result = ''
    for i in range(n):
        result += random.choice('abcdefghijklmnopqrstuvwxyz')
    return result


items = weave.save(
    ops_arrow.to_arrow(sorted([
        {
            "pred": random_suggestion(7),
            "time": random_datetime_in_range(),
            "user": random.choice(users),
            "version": random.choice(model_versions),
            "accepted": random.choice([1, 1, 1, 0, 0]),
            "metric": random.uniform(0, 1)
        } 
        for _ in range(num_requests)
    ], key=lambda d: d['time']))
)

In [ ]:
from weave.legacy.weave.panels.panel_plot import Signals, AxisSelections, Plot
from weave.legacy.weave import weave_internal


domain_node = weave_internal.const(None)
#domain_node = weave_internal.save(None)

board = weave.legacy.weave.panels.Board(
    vars={
        'predictions': items,
        'domain': domain_node,
    },
    panels=[
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, domain: Plot(
                            predictions,
                            x=lambda row: row["time"],
                            y=lambda row: row["metric"],
                            label=lambda row: row['user'],
                            mark="point",
                            domain_x=domain,
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=12, h=12)
        ),
        weave.legacy.weave.panels.BoardPanel(
            lambda predictions, domain: Plot(
                            predictions,
                            x=lambda row: row["time"],
                            y=lambda row: row["metric"] * 2,
                            label=lambda row: row['version'],
                            mark="point",
                            domain_x=domain,
                        ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=12, h=12)
        ),        
    ]
)
board